# Data Preprocessing

## Data Acquisition

This is the "acquisition" of data for the country selector project. I do not intend to publish my work in hopes for any material benefits, however, I will list here all the data resources I have used for this project.

Firstly, I would like to take you through my thought process. At first, I wanted to use a Kaggle data set upon which I could try my hands at clustering. This would have been done through the [world happiness report dataset](https://www.kaggle.com/datasets/mathurinache/world-happiness-report). But I wanted to build it as a project that would be useful to many people in general. So I came up with the idea of a country selector. I have all these clusters of countries which have a lot of things in common; I check what those things are and I use them as "group features". These group features will then be presented to the user as a choice and then the user will see what group of countries scores the best in those areas, hence giving him an idea of what countries should he look more into.

All this having been said, I have done a fast (I still wanted to keep this a small project) research in order to see what are the most important criteria when choosing another place to live. I have documented this small endeavour in [this file](research.txt). Keeping these measures in mind, I browsed the web for data about countries' healthcare system, finance, climate, laws etc. and downloaded them. Again, keeping it short, since data was not available on one site, I just downloaded it instead of learning how to use different APIs. 

I have used the following websites to download my data and I give them full credit for this data:

https://climatedata.worldbank.org

https://worldpopulationreview.com

https://databank.worldbank.org

https://www.kaggle.com/datasets/mathurinache/world-happiness-report


## Data Preprocessing

This notebook will deal with transforming all the data obtained into one smooth dataset. Therefore here I will combine all these datasets into one. I will drop all the countries which have less than 70% of necessary data or (if possible) find a solution to impute missing data. The data is also not similar in terms of timeline. Therefore, if the data is available, I will select the most recent one. All the data I have downloaded is at most 4 years old (2018) at the moment of this notebook's creation.

I will also see if I can transform data values so that they are on similar scales, since it should (in theory) be better for clustering. I will apply other aggregations and replacement where I see fit.

Therefore there will be 4 steps, 1 for each type of data. The 5th an final step will be getting all these datasets cleaned and into one useful dataset to be used in the main part.

Without further ado, let's wrangle some data.

In [192]:
import pandas as pd
import numpy as np
import re
import sys
# Insert at 1, 0 is the script path
sys.path.insert(1, './python-scripts/development')
from preprocessing_functions import categorize_gdp_per_capita_value, preprocess_world_bank_data, categorize_secure_internet_servers_per_million_people
from preprocessing_variables import data_scaler, data_scaler_type
import xarray as xr

## Part one - world happiness report

I will now see all the useful data I can extract from the world happiness report.

In [193]:
world_happiness = pd.read_csv("../data/world-happiness-2022.csv", index_col="RANK", decimal=",")
world_happiness.head()

,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
RANK,,,,,,,,,,,
1,Finland,7.821,7.886,7.756,2.518,1.892,1.258,0.775,0.736,0.109,0.534
2,Denmark,7.636,7.710,7.563,2.226,1.953,1.243,0.777,0.719,0.188,0.532
3,Iceland,7.557,7.651,7.464,2.320,1.936,1.320,0.803,0.718,0.270,0.191
4,Switzerland,7.512,7.586,7.437,2.153,2.026,1.226,0.822,0.677,0.147,0.461
5,Netherlands,7.415,7.471,7.359,2.137,1.945,1.206,0.787,0.651,0.271,0.419


In [194]:
# See if there are any null values etc.
world_happiness.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 1 to 147
Data columns (total 11 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Country                                     147 non-null    object 
 1   Happiness score                             146 non-null    float64
 2   Whisker-high                                146 non-null    float64
 3   Whisker-low                                 146 non-null    float64
 4   Dystopia (1.83) + residual                  146 non-null    float64
 5   Explained by: GDP per capita                146 non-null    float64
 6   Explained by: Social support                146 non-null    float64
 7   Explained by: Healthy life expectancy       146 non-null    float64
 8   Explained by: Freedom to make life choices  146 non-null    float64
 9   Explained by: Generosity                    146 non-null    float64
 10  Explained by: 

In [195]:
# Check entries with null values
world_happiness[pd.isnull(world_happiness).any(axis=1)]

,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
RANK,,,,,,,,,,,
147,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
# It looks like the 147th entry is not an actual country, so I can drop it.
world_happiness = world_happiness.drop(index=147)
world_happiness.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 1 to 146
Data columns (total 11 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Country                                     146 non-null    object 
 1   Happiness score                             146 non-null    float64
 2   Whisker-high                                146 non-null    float64
 3   Whisker-low                                 146 non-null    float64
 4   Dystopia (1.83) + residual                  146 non-null    float64
 5   Explained by: GDP per capita                146 non-null    float64
 6   Explained by: Social support                146 non-null    float64
 7   Explained by: Healthy life expectancy       146 non-null    float64
 8   Explained by: Freedom to make life choices  146 non-null    float64
 9   Explained by: Generosity                    146 non-null    float64
 10  Explained by: 

In [197]:
# I will also drop the whisker high, whisker low, and dystopia features as being irrelevant for the state of our project
world_happiness = world_happiness.drop(columns=["Whisker-high", "Whisker-low", "Dystopia (1.83) + residual"])
world_happiness.head()

,Country,Happiness score,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
RANK,,,,,,,,
1,Finland,7.821,1.892,1.258,0.775,0.736,0.109,0.534
2,Denmark,7.636,1.953,1.243,0.777,0.719,0.188,0.532
3,Iceland,7.557,1.936,1.320,0.803,0.718,0.270,0.191
4,Switzerland,7.512,2.026,1.226,0.822,0.677,0.147,0.461
5,Netherlands,7.415,1.945,1.206,0.787,0.651,0.271,0.419


In [198]:
# Furthermore, we will rename some of the features so we can use them more easily
world_happiness.rename(columns=lambda c: c.replace("Explained by: ", "").replace(" ", "_").lower(), inplace=True)
world_happiness.head()

,country,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
RANK,,,,,,,,
1,Finland,7.821,1.892,1.258,0.775,0.736,0.109,0.534
2,Denmark,7.636,1.953,1.243,0.777,0.719,0.188,0.532
3,Iceland,7.557,1.936,1.320,0.803,0.718,0.270,0.191
4,Switzerland,7.512,2.026,1.226,0.822,0.677,0.147,0.461
5,Netherlands,7.415,1.945,1.206,0.787,0.651,0.271,0.419


In [199]:
# Another important step is scaling.
# I will use a either min max scaling, either standard scaling so that all features are between 0 and 1 and they 
# keep their impact respective to their groups. The value for the scaler is set in preprocessing_variables.py

# Get numerical columns for the world happiness dataset
numerical_columns_w_h = world_happiness.columns.copy().drop("country")

world_happiness[numerical_columns_w_h] = data_scaler.fit_transform(world_happiness[numerical_columns_w_h])
world_happiness.head()

,country,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
RANK,,,,,,,,
1,Finland,1.000000,0.856496,0.953030,0.822718,0.994595,0.232906,0.909710
2,Denmark,0.965848,0.884110,0.941667,0.824841,0.971622,0.401709,0.906303
3,Iceland,0.951265,0.876415,1.000000,0.852442,0.970270,0.576923,0.325383
4,Switzerland,0.942957,0.917157,0.928788,0.872611,0.914865,0.314103,0.785349
5,Netherlands,0.925051,0.880489,0.913636,0.835456,0.879730,0.579060,0.713799


## Part 2 - databank datasets

This is the biggest chunk of the work. There are 10 datasets here, each of which has data for years 2020 and 2021 for every country. I will choose data from 2020 where it is not available for 2021. If no data is available, then data for that country will be null.

I will take all the datasets and try to compile them into one. For countries which miss most of the values I will drop them. For columns which do not have most of the values, I will drop them.

In [200]:
economy_stats = pd.read_excel("../data/Economy.xlsx", na_values="..")
economy_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Adjusted net national income (current US$),NY.ADJ.NNTY.CD,1.845879e+10,NaN
1,Afghanistan,AFG,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,-1.559312e+01,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,5.167479e+02,NaN
3,Afghanistan,AFG,GNI per capita (constant 2015 US$),NY.GNP.PCAP.KD,NaN,NaN
4,Albania,ALB,Adjusted net national income (current US$),NY.ADJ.NNTY.CD,1.193938e+10,NaN


In [201]:
economy_stats = preprocess_world_bank_data(economy_stats)
economy_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   country_name                                195 non-null    object 
 1   Adjusted net national income (current US$)  170 non-null    float64
 2   Current account balance (% of GDP)          167 non-null    float64
 3   GDP per capita (current US$)                194 non-null    float64
 4   GNI per capita (constant 2015 US$)          128 non-null    float64
dtypes: float64(4), object(1)
memory usage: 7.7+ KB


1. It looks like for almost all countries I have the GDP per capita. I will check which country does not and through a quick google search see if I can find it.
2. Since all this is capital related, it makes sense that to fill the adjusted net national income and current account balance, I can group countries by GDP per capita and then fill missing values with the mean of the group.
3. Since for GNI per capita there are fewer than 70% of the values (65%), I will drop it.

In [202]:
# Check what kind of values we have so we can see what kind of groups we can create
economy_stats["GDP per capita (current US$)"].describe()

count       194.000000
mean      15348.067094
std       23405.739594
min         238.990726
25%        2169.762588
50%        5467.472829
75%       17714.185985
max      173688.189360
Name: GDP per capita (current US$), dtype: float64

In [203]:
# Drop GNI per capita column
economy_stats = economy_stats.drop(columns="GNI per capita (constant 2015 US$)")

# Check what country does not have the GDP per capita value assigned
economy_stats[economy_stats["GDP per capita (current US$)"].isnull()]

series_name,country_name,Adjusted net national income (current US$),Current account balance (% of GDP),GDP per capita (current US$)
162,South Sudan,NaN,-35.007727,NaN


In [204]:
# According to https://statisticstimes.com/economy/country/south-sudan-gdp-per-capita.php
# and https://knoema.com/atlas/South-Sudan/GDP-per-capita
# I can conclude that the GDP per capita for Sudan cane be set at 296
economy_stats.loc[162, "GDP per capita (current US$)"] = 296.0
economy_stats[economy_stats["GDP per capita (current US$)"].isnull()]

series_name,country_name,Adjusted net national income (current US$),Current account balance (% of GDP),GDP per capita (current US$)


In [205]:
# Get numerical columns for the economy stats dataset
numerical_columns_e_s = economy_stats.columns.copy().drop("country_name")

# Fill NA values with GDP per capita group mean
economy_stats[numerical_columns_e_s] = economy_stats.groupby(
    economy_stats["GDP per capita (current US$)"].apply(
        lambda x: categorize_gdp_per_capita_value)).transform(lambda x: x.fillna(x.mean()))
economy_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 4 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   country_name                                195 non-null    object 
 1   Adjusted net national income (current US$)  195 non-null    float64
 2   Current account balance (% of GDP)          195 non-null    float64
 3   GDP per capita (current US$)                195 non-null    float64
dtypes: float64(3), object(1)
memory usage: 6.2+ KB


C:\Users\Andrei\AppData\Local\Temp\ipykernel_13376\774526914.py:5: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  economy_stats[numerical_columns_e_s] = economy_stats.groupby(


In [206]:
# Again, I will use scaling so that all features keep their impact respective to their groups
economy_stats[numerical_columns_e_s] = data_scaler.fit_transform(economy_stats[numerical_columns_e_s])
economy_stats.head()

series_name,country_name,Adjusted net national income (current US$),Current account balance (% of GDP),GDP per capita (current US$)
0,Afghanistan,0.001026,0.277733,0.001601
1,Albania,0.000657,0.372249,0.028868
2,Algeria,0.006776,0.320042,0.017687
3,American Samoa,0.021364,0.457369,0.072678
4,Angola,0.002009,0.516524,0.008862


In [207]:
education_stats = pd.read_excel("../data/education.xlsx", na_values="..")
education_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,NaN,NaN
1,Afghanistan,AFG,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN
2,Afghanistan,AFG,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,37.266041
3,Afghanistan,AFG,"Educational attainment, at least Bachelor's or...",SE.TER.CUAT.BA.ZS,NaN,3.067980
4,Afghanistan,AFG,"Educational attainment, at least completed low...",SE.SEC.CUAT.LO.ZS,NaN,11.631920


In [208]:
education_stats = preprocess_world_bank_data(education_stats)
education_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 5 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   country_name                                                                                        69 non-null     object 
 1   Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)   38 non-null     float64
 2   Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)  38 non-null     float64
 3   Government expenditure on education, total (% of GDP)                                               35 non-null     float64
 4   Literacy rate, adult total (% of people ages 15 and above)                                          16 non-null     flo

It looks like this data is mostly null, hence not useful at all. This means that I will need to find another source for education data. I will let this be for now. After processing all the available data I will see what kind of data I need and see if I can get it.

In [209]:
employment_stats = pd.read_excel("../data/employment.xlsx", na_values="..")
employment_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Adequacy of social protection and labor progra...,per_allsp.adq_pop_tot,NaN,NaN
1,Afghanistan,AFG,"Employers, total (% of total employment) (mode...",SL.EMP.MPYR.ZS,NaN,NaN
2,Afghanistan,AFG,Employment in industry (% of total employment)...,SL.IND.EMPL.ZS,NaN,NaN
3,Afghanistan,AFG,Employment in agriculture (% of total employme...,SL.AGR.EMPL.ZS,NaN,NaN
4,Afghanistan,AFG,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.NE.ZS,36.709999,NaN


In [210]:
employment_stats = preprocess_world_bank_data(employment_stats)
employment_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   country_name                                                        99 non-null     object 
 1   Employment to population ratio, 15+, total (%) (national estimate)  95 non-null     float64
 2   Unemployment, total (% of total labor force) (national estimate)    99 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.4+ KB


It looks like only 2 columns were non null. I can use this data. Furthermore I can see that they mean more or less the same thing. Therefore I will keep the second statistic (unemployment rate).

I will scale it and then reverse it (take 1 - scaled_value) so that countries with low unemployment score low in this. I will do this because all of the statistics I have until now are positive functions, meaning that the bigger the value, the better the country in that particular area.

This is just my assumption that if we have the same measurements for each featue it will lead to better clustering.

In [211]:
employment_stats = employment_stats.drop(columns="Employment to population ratio, 15+, total (%) (national estimate)")
# Check if there are any more null values
employment_stats[employment_stats.isna().any(axis=1)]

series_name,country_name,"Unemployment, total (% of total labor force) (national estimate)"


In [212]:
# Get numerical columns for the world happiness dataset
numerical_columns_em_s = employment_stats.columns.copy().drop("country_name")

# Again, I will use scaling so that all features keep their impact respective to their groups
employment_stats[numerical_columns_em_s] = data_scaler.fit_transform(employment_stats[numerical_columns_em_s])
employment_stats.head()

series_name,country_name,"Unemployment, total (% of total labor force) (national estimate)"
0,Afghanistan,0.397868
1,Argentina,0.389271
2,Armenia,0.620358
3,Australia,0.170908
4,Austria,0.179505


In [213]:
# Reverse statistic so high-value = good, low-value = bad; also rename column
employment_stats[numerical_columns_em_s] = 1.0 - employment_stats[numerical_columns_em_s]
employment_stats.rename(columns={'Unemployment, total (% of total labor force) (national estimate)': 'employment_rate_labor_force'})
employment_stats.head()

series_name,country_name,"Unemployment, total (% of total labor force) (national estimate)"
0,Afghanistan,0.602132
1,Argentina,0.610729
2,Armenia,0.379642
3,Australia,0.829092
4,Austria,0.820495


In [214]:
financial_indicators = pd.read_excel("../data/financial-indicators.xlsx", na_values="..")
financial_indicators.head(20)

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN
1,Afghanistan,AFG,Consumer price index (2010 = 100),FP.CPI.TOTL,NaN,NaN
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN
3,Afghanistan,AFG,"Listed domestic companies, total",CM.MKT.LDOM.NO,NaN,NaN
4,Albania,ALB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN
5,Albania,ALB,Consumer price index (2010 = 100),FP.CPI.TOTL,120.978912,123.448662
6,Albania,ALB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1.620887,2.041472
7,Albania,ALB,"Listed domestic companies, total",CM.MKT.LDOM.NO,NaN,NaN
8,Algeria,DZA,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN
9,Algeria,DZA,Consumer price index (2010 = 100),FP.CPI.TOTL,155.017250,166.218895


In [215]:
financial_indicators = preprocess_world_bank_data(financial_indicators)
financial_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 4 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   country_name                           164 non-null    object 
 1   Consumer price index (2010 = 100)      160 non-null    float64
 2   Inflation, consumer prices (annual %)  160 non-null    float64
 3   Listed domestic companies, total       69 non-null     float64
dtypes: float64(3), object(1)
memory usage: 5.2+ KB


I will drop 'listed domestic companies total'. As for the other 2 indexes, I will search which countries do not have it and will search for the values on the web.

Furthermore, after scaling, I will also revert the values (take the 1 - scaled_value) so that a value increase represents a good change.

In [216]:
financial_indicators.loc[(financial_indicators['Consumer price index (2010 = 100)'].isnull()) 
                         | (financial_indicators['Inflation, consumer prices (annual %)'].isnull())]

series_name,country_name,Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)","Listed domestic companies, total"
10,Barbados,NaN,NaN,15.0
27,Cayman Islands,NaN,NaN,4.0
68,"Iran, Islamic Rep.",NaN,NaN,367.0
109,Nigeria,NaN,NaN,177.0


In [217]:
# Complete with values found on the web
financial_indicators.loc[10, 'Consumer price index (2010 = 100)'] = 212.3
financial_indicators.loc[10, 'Inflation, consumer prices (annual %)'] = 4.1003

financial_indicators.loc[27, 'Consumer price index (2010 = 100)'] = 121
financial_indicators.loc[27, 'Inflation, consumer prices (annual %)'] = -0.6

financial_indicators.loc[68, 'Consumer price index (2010 = 100)'] = 415
financial_indicators.loc[68, 'Inflation, consumer prices (annual %)'] = 39.9

financial_indicators.loc[109, 'Consumer price index (2010 = 100)'] = 439
financial_indicators.loc[109, 'Inflation, consumer prices (annual %)'] = 11.4

In [218]:
# Drop unuseful column
financial_indicators = financial_indicators.drop(columns="Listed domestic companies, total")

# Get numerical columns for the world happiness dataset
numerical_columns_f_i = financial_indicators.columns.copy().drop("country_name")

# Again, I will use scaling so that all features keep their impact respective to their groups
financial_indicators[numerical_columns_f_i] = data_scaler.fit_transform(financial_indicators[numerical_columns_f_i])
financial_indicators.head()

series_name,country_name,Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)"
0,Albania,0.001183,0.007368
1,Algeria,0.003288,0.016638
2,Antigua and Barbuda,0.000943,0.007407
3,Armenia,0.002004,0.016565
4,Australia,0.001224,0.008839


In [219]:
# Reverse statistic so high-value = good, low-value = bad; also rename column
financial_indicators[numerical_columns_f_i] = 1.0 - financial_indicators[numerical_columns_f_i]
financial_indicators.rename(columns={'Consumer price index (2010 = 100)': 'inverse_of_consumer_price_', 
                                     'Inflation, consumer prices (annual %)': 'inverse_of_inflation'})
financial_indicators.head()

series_name,country_name,Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)"
0,Albania,0.998817,0.992632
1,Algeria,0.996712,0.983362
2,Antigua and Barbuda,0.999057,0.992593
3,Armenia,0.997996,0.983435
4,Australia,0.998776,0.991161


In [220]:
health_stats = pd.read_excel("../data/health.xlsx", na_values="..")
health_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,"Current health expenditure per capita, PPP (cu...",SH.XPD.CHEX.PP.CD,NaN,NaN
1,Afghanistan,AFG,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,NaN,NaN
2,Afghanistan,AFG,Mortality caused by road traffic injury (per 1...,SH.STA.TRAF.P5,NaN,NaN
3,Afghanistan,AFG,Out-of-pocket expenditure per capita (current ...,SH.XPD.OOPC.PC.CD,NaN,NaN
4,Afghanistan,AFG,People with basic handwashing facilities inclu...,SH.STA.HYGN.ZS,38.11505,NaN


In [221]:
health_stats = preprocess_world_bank_data(health_stats)
health_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 3 columns):
 #   Column                                                                               Non-Null Count  Dtype  
---  ------                                                                               --------------  -----  
 0   country_name                                                                         216 non-null    object 
 1   People with basic handwashing facilities including soap and water (% of population)  79 non-null     float64
 2   Population growth (annual %)                                                         216 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.2+ KB


It looks like we can use the 'population growth' value, but not the other one. Hence, we will drop the unuseful one and keep the useful one.

In [222]:
# Drop columns with too many null values
health_stats = health_stats.drop(
    columns="People with basic handwashing facilities including soap and water (% of population)")

# Get numerical columns for the world happiness dataset
numerical_columns_h_s = health_stats.columns.copy().drop("country_name")

# Again, I will use scaling so that all features keep their impact respective to their groups
health_stats[numerical_columns_h_s] = data_scaler.fit_transform(health_stats[numerical_columns_h_s])
health_stats.head()

series_name,country_name,Population growth (annual %)
0,Afghanistan,0.730947
1,Albania,0.203302
2,Algeria,0.645278
3,American Samoa,0.270417
4,Andorra,0.336833


In [223]:
infrastructure_stats = pd.read_excel("../data/infrastructure.xlsx", na_values="..")
infrastructure_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Fixed broadband subscriptions (per 100 people),IT.NET.BBND.P2,0.068254,NaN
1,Afghanistan,AFG,Rail lines (total route-km),IS.RRS.TOTL.KM,NaN,NaN
2,Afghanistan,AFG,Secure Internet servers (per 1 million people),IT.NET.SECR.P6,34.987363,NaN
3,Afghanistan,AFG,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
4,Albania,ALB,Fixed broadband subscriptions (per 100 people),IT.NET.BBND.P2,17.684951,NaN


In [224]:
infrastructure_stats = preprocess_world_bank_data(infrastructure_stats)
infrastructure_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 3 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   country_name                                    215 non-null    object 
 1   Fixed broadband subscriptions (per 100 people)  201 non-null    float64
 2   Secure Internet servers (per 1 million people)  214 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.2+ KB


It looks like we have 2 infrastructure indicators that we can use. For the column with the missing value I will search manually for the value, but for the one with 14 missing value, I will fill null values based on group means.

In [225]:
infrastructure_stats.loc[infrastructure_stats["Secure Internet servers (per 1 million people)"].isnull()]

series_name,country_name,Fixed broadband subscriptions (per 100 people),Secure Internet servers (per 1 million people)
150,Papua New Guinea,0.234715,NaN


In [226]:
# Fill with found value
infrastructure_stats.loc[150, "Secure Internet servers (per 1 million people)"] = 56.74
infrastructure_stats.loc[infrastructure_stats["Secure Internet servers (per 1 million people)"].isnull()]

series_name,country_name,Fixed broadband subscriptions (per 100 people),Secure Internet servers (per 1 million people)


In [227]:
# Get numerical columns for the economy stats dataset
numerical_columns_i_s = infrastructure_stats.columns.copy().drop("country_name")

# Fill NA values with GDP per capita group mean
infrastructure_stats[numerical_columns_i_s] = infrastructure_stats.groupby(
    infrastructure_stats["Fixed broadband subscriptions (per 100 people)"].apply(
        lambda x: categorize_secure_internet_servers_per_million_people)).transform(lambda x: x.fillna(x.mean()))
infrastructure_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 3 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   country_name                                    215 non-null    object 
 1   Fixed broadband subscriptions (per 100 people)  215 non-null    float64
 2   Secure Internet servers (per 1 million people)  215 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.2+ KB


C:\Users\Andrei\AppData\Local\Temp\ipykernel_13376\1489755229.py:5: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  infrastructure_stats[numerical_columns_i_s] = infrastructure_stats.groupby(


In [228]:
# Again, I will use scaling so that all features keep their impact respective to their groups
infrastructure_stats[numerical_columns_i_s] = data_scaler.fit_transform(infrastructure_stats[numerical_columns_i_s])
infrastructure_stats.head()

series_name,country_name,Fixed broadband subscriptions (per 100 people),Secure Internet servers (per 1 million people)
0,Afghanistan,0.001095,0.000047
1,Albania,0.283604,0.001194
2,Algeria,0.138618,0.000065
3,American Samoa,0.265331,0.000293
4,Andorra,0.767940,0.013116


In [229]:
population_and_environment_stats = pd.read_excel("../data/population-and-environment.xlsx", na_values="..")
population_and_environment_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,97.699997,NaN
1,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN
2,Afghanistan,AFG,Electricity production from coal sources (% of...,EG.ELC.COAL.ZS,NaN,NaN
3,Afghanistan,AFG,Land area (sq. km),AG.LND.TOTL.K2,652860.000000,652860.0
4,Afghanistan,AFG,Forest area (% of land area),AG.LND.FRST.ZS,1.850994,NaN


In [230]:
population_and_environment_stats = preprocess_world_bank_data(population_and_environment_stats)
population_and_environment_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 6 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   country_name                                         216 non-null    object 
 1   Access to electricity (% of population)              215 non-null    float64
 2   Forest area (% of land area)                         210 non-null    float64
 3   Land area (sq. km)                                   216 non-null    float64
 4   Population density (people per sq. km of land area)  216 non-null    float64
 5   Urban population (% of total population)             214 non-null    float64
dtypes: float64(5), object(1)
memory usage: 10.2+ KB


Because of the low number of null values we can impute them value by value

In [231]:
population_and_environment_stats[population_and_environment_stats['Access to electricity (% of population)'].isnull()]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)
3,American Samoa,NaN,85.65,200.0,275.985,87.153


In [232]:
# Impute with found value on the web
population_and_environment_stats.loc[3, 'Access to electricity (% of population)'] = 99.2
population_and_environment_stats[population_and_environment_stats['Access to electricity (% of population)'].isnull()]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)


In [233]:
population_and_environment_stats[population_and_environment_stats['Forest area (% of land area)'].isnull()]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)
39,Channel Islands,100.0,NaN,198.000,878.075758,30.963
75,Gibraltar,100.0,NaN,10.000,3369.100000,100.000
86,"Hong Kong SAR, China",100.0,NaN,1050.000,7125.523810,100.000
116,"Macao SAR, China",100.0,NaN,32.900,19736.838906,100.000
129,Monaco,100.0,NaN,2.027,19360.631475,100.000
136,Nauru,100.0,NaN,20.000,541.700000,100.000


In [234]:
# Impute with found values on the web
population_and_environment_stats.loc[39, 'Forest area (% of land area)'] = 5.15
population_and_environment_stats.loc[75, 'Forest area (% of land area)'] = 0
population_and_environment_stats.loc[86, 'Forest area (% of land area)'] = 56
population_and_environment_stats.loc[116, 'Forest area (% of land area)'] = 1.5
population_and_environment_stats.loc[129, 'Forest area (% of land area)'] = 0
population_and_environment_stats.loc[136, 'Forest area (% of land area)'] = 0

population_and_environment_stats[population_and_environment_stats['Forest area (% of land area)'].isnull()]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)


In [235]:
population_and_environment_stats[population_and_environment_stats['Urban population (% of total population)'].isnull()]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)
61,Eritrea,52.171097,10.448119,101000.0,35.113139,NaN
182,St. Martin (French part),100.000000,24.800000,50.0,773.180000,NaN


In [236]:
# Impute with found values on the web
population_and_environment_stats.loc[61, 'Urban population (% of total population)'] = 40.71
population_and_environment_stats.loc[182, 'Urban population (% of total population)'] = 0

population_and_environment_stats[population_and_environment_stats['Urban population (% of total population)'].isnull()]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)


In [237]:
# Check if there are any more null values
population_and_environment_stats[population_and_environment_stats.isna().any(axis=1)]

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)


In [238]:
# Get numerical columns for the world happiness dataset
numerical_columns_p_e_s = population_and_environment_stats.columns.copy().drop("country_name")

# Again, I will use scaling so that all features keep their impact respective to their groups
population_and_environment_stats[numerical_columns_p_e_s] = data_scaler.fit_transform(
    population_and_environment_stats[numerical_columns_p_e_s])
population_and_environment_stats.head()

series_name,country_name,Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)
0,Afghanistan,0.975204,0.019002,0.039865,0.003014,0.26026
1,Albania,1.000000,0.295569,0.001673,0.005241,0.62112
2,Algeria,0.997888,0.008400,0.145433,0.000926,0.73733
3,American Samoa,0.991375,0.879254,0.000012,0.013976,0.87153
4,Andorra,1.000000,0.349469,0.000029,0.008322,0.87916


In [239]:
poverty_stats = pd.read_excel("../data/poverty.xlsx", na_values="..")
poverty_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Multidimensional poverty headcount ratio (% of...,SI.POV.MDIM,49.4,NaN
1,Albania,ALB,Multidimensional poverty headcount ratio (% of...,SI.POV.MDIM,43.4,NaN
2,Algeria,DZA,Multidimensional poverty headcount ratio (% of...,SI.POV.MDIM,NaN,NaN
3,American Samoa,ASM,Multidimensional poverty headcount ratio (% of...,SI.POV.MDIM,NaN,NaN
4,Andorra,AND,Multidimensional poverty headcount ratio (% of...,SI.POV.MDIM,NaN,NaN


In [240]:
poverty_stats = preprocess_world_bank_data(poverty_stats)
poverty_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   country_name                                                      35 non-null     object 
 1   Multidimensional poverty headcount ratio (% of total population)  35 non-null     float64
dtypes: float64(1), object(1)
memory usage: 688.0+ bytes


Again, insufficient data, because there are only 35 countries with this data. Will do the same as stated above.

In [241]:
private_sector_stats = pd.read_excel("../data/private-sector.xlsx", na_values="..")
private_sector_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Cost of business start-up procedures (% of GNI...,IC.REG.COST.PC.ZS,NaN,NaN
1,Afghanistan,AFG,Ease of doing business score (0 = lowest perfo...,IC.BUS.DFRN.XQ,NaN,NaN
2,Afghanistan,AFG,Labor tax and contributions (% of commercial p...,IC.TAX.LABR.CP.ZS,NaN,NaN
3,Albania,ALB,Cost of business start-up procedures (% of GNI...,IC.REG.COST.PC.ZS,NaN,NaN
4,Albania,ALB,Ease of doing business score (0 = lowest perfo...,IC.BUS.DFRN.XQ,NaN,NaN


In [242]:
private_sector_stats = preprocess_world_bank_data(private_sector_stats)
private_sector_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_name  0 non-null      object
dtypes: object(1)
memory usage: 124.0+ bytes


Again, insufficient data. Will do the same as stated above.

In [243]:
public_sector_stats = pd.read_excel("../data/public-sector.xlsx", na_values="..")
public_sector_stats.head()

,Country Name,Country Code,Series Name,Series Code,2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,CPIA financial sector rating (1=low to 6=high),IQ.CPA.FINS.XQ,1.5,NaN
1,Afghanistan,AFG,CPIA gender equality rating (1=low to 6=high),IQ.CPA.GNDR.XQ,1.5,NaN
2,Afghanistan,AFG,CPIA policies for social inclusion/equity clus...,IQ.CPA.SOCI.XQ,2.7,NaN
3,Afghanistan,AFG,CPIA property rights and rule-based governance...,IQ.CPA.PROP.XQ,2.0,NaN
4,Afghanistan,AFG,CPIA quality of public administration rating (...,IQ.CPA.PADM.XQ,2.5,NaN


In [244]:
public_sector_stats = preprocess_world_bank_data(public_sector_stats)
public_sector_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 6 columns):
 #   Column                                                                       Non-Null Count  Dtype  
---  ------                                                                       --------------  -----  
 0   country_name                                                                 73 non-null     object 
 1   CPIA financial sector rating (1=low to 6=high)                               73 non-null     float64
 2   CPIA gender equality rating (1=low to 6=high)                                73 non-null     float64
 3   CPIA policies for social inclusion/equity cluster average (1=low to 6=high)  73 non-null     float64
 4   CPIA property rights and rule-based governance rating (1=low to 6=high)      73 non-null     float64
 5   CPIA quality of public administration rating (1=low to 6=high)               73 non-null     float64
dtypes: float64(5), object(1)
memory usage: 3.5+ K

In [245]:
# Rename new columns
public_sector_stats.rename(columns=lambda c: re.sub(r' \(.*\)', '', c), inplace=True)
public_sector_stats.rename(columns=lambda c: re.sub(r'CPIA ', '', c), inplace=True)
public_sector_stats.rename(columns=lambda c: c.replace(" ", "_").lower(), inplace=True)
public_sector_stats.head()

series_name,country_name,financial_sector_rating,gender_equality_rating,policies_for_social_inclusion/equity_cluster_average,property_rights_and_rule-based_governance_rating,quality_of_public_administration_rating
0,Afghanistan,1.5,1.5,2.7,2.0,2.5
1,Bangladesh,2.5,3.0,3.3,2.5,2.0
2,Benin,2.5,3.5,3.5,3.5,3.0
3,Bhutan,3.0,4.0,4.0,4.0,4.0
4,Burkina Faso,3.0,3.5,3.6,3.0,3.0


In [246]:
# Get numerical columns for the world happiness dataset
numerical_columns_pu_s_s = public_sector_stats.columns.copy().drop("country_name")

# Again, I will use scaling so that all features keep their impact respective to their groups
public_sector_stats[numerical_columns_pu_s_s] = data_scaler.fit_transform(public_sector_stats[numerical_columns_pu_s_s])
public_sector_stats.head()

series_name,country_name,financial_sector_rating,gender_equality_rating,policies_for_social_inclusion/equity_cluster_average,property_rights_and_rule-based_governance_rating,quality_of_public_administration_rating
0,Afghanistan,0.166667,0.000000,0.428571,0.333333,0.500000
1,Bangladesh,0.500000,0.500000,0.642857,0.500000,0.333333
2,Benin,0.500000,0.666667,0.714286,0.833333,0.666667
3,Bhutan,0.666667,0.833333,0.892857,1.000000,1.000000
4,Burkina Faso,0.666667,0.666667,0.750000,0.666667,0.666667


Up until this point we have 7 data sets which yielded results: economy_stats, public_sector_stats, financial_indicators, health_stats and population_and_environment_stats, employment_stats, infrastructure_stats.

The other 3 data sets did not yield any useful result: education_stats, poverty_stats, private_sector_stats.

I will search for this other type of data and see if I can find it quickly. If I cannot, I will leave it be and work with the available data.

## Part 3 - climate data sets

This consists of 2 files: precipitation and mean temperature.

In [247]:
precipitation_ds = xr.open_dataset('../data/precipitation.nc')
precipitation_stats = precipitation_ds.to_dataframe()
precipitation_stats.head(20)

climatology-pr-annual-mean  lon_bnds  lat_bnds
time       lat    lon     bnds                                                
1991-01-16 -89.75 -179.75 0                            NaN    -180.0     -90.0
                          1                            NaN    -179.5     -89.5
                  -179.25 0                            NaN    -179.5     -90.0
                          1                            NaN    -179.0     -89.5
                  -178.75 0                            NaN    -179.0     -90.0
                          1                            NaN    -178.5     -89.5
                  -178.25 0                            NaN    -178.5     -90.0
                          1                            NaN    -178.0     -89.5
                  -177.75 0                            NaN    -178.0     -90.0
                          1                            NaN    -177.5     -89.5
                  -177.25 0                            NaN    -177.5     -90.0
                          1                            NaN    -177.0     -89.5
                  -176.75 0                            NaN    -177.0     -90.0
                          1                            NaN    -176.5     -89.5
                  -176.25 0                            NaN    -176.5     -90.0
                          1                            NaN    -176.0     -89.5
                  -175.75 0                            NaN    -176.0     -90.0
                          1                            NaN    -175.5     -89.5
                  -175.25 0                            NaN    -175.5     -90.0
                          1                            NaN    -175.0     -89.5

It looks like this data is based on latitude and not on countries. It will be the same for temperature as I have downloaded from the same website. This is useless.

Therefore I decided to go on without it.

After a brief search on the web so get this data, I could not find something quick and free. because I already have some good info and I want to keep this project short, I will not proceeed any further. There will be no information about precipitation and temperature.

## Part 4 - crime statistics

This consists of 1 file: crime-rates.json

In [248]:
crime_rates = pd.read_json("../data/crime-rates.json")
crime_rates.head(10)

,ranking,country,crimeIndex,pop2022
0,1,Venezuela,83.76,29266.991
1,2,Papua New Guinea,80.79,9292.169
2,3,South Africa,76.86,60756.135
3,4,Afghanistan,76.31,40754.388
4,5,Honduras,74.54,10221.247
5,6,Trinidad And Tobago,71.63,1406.585
6,7,Guyana,68.74,794.045
7,8,El Salvador,67.79,6550.389
8,9,Brazil,67.49,215353.593
9,10,Jamaica,67.42,2985.094


In [249]:
# Drop useless columns ranking and pop2022
crime_rates = crime_rates.drop(columns=['ranking', 'pop2022'])

# Again, I will use scaling so that all features keep their impact respective to their groups
crime_rates[['crimeIndex']] = data_scaler.fit_transform(crime_rates[['crimeIndex']])
crime_rates.head()

,country,crimeIndex
0,Venezuela,1.000000
1,Papua New Guinea,0.958537
2,South Africa,0.903672
3,Afghanistan,0.895993
4,Honduras,0.871283


In [250]:
# Reverse statistic so high-value = good, low-value = bad; also rename column
crime_rates['crimeIndex'] = 1.0 - crime_rates['crimeIndex']
crime_rates.rename(columns={'crimeIndex': 'opposite_of_crime_index'}, inplace=True)
crime_rates.head()

,country,opposite_of_crime_index
0,Venezuela,1.110223e-16
1,Papua New Guinea,4.146307e-02
2,South Africa,9.632835e-02
3,Afghanistan,1.040067e-01
4,Honduras,1.287170e-01


## Step 5 - putting them all together

This is the most important stept. We now have 1 + 7 + 0 + 1 = 9 data sets, each having different statistics whose values are from 0 = bad to 1 = great.

Each data set has statistics for a certain number of countries. There is a total of 195 countries in the world (according to Google). I would like to be able to include at least 70% of them in this project. Therefore:

- I will drop any data set with less than 136 countries
- If I consider I will have too few features for the 70% margin, I will drop it to 60%

I will see what are those countries that have statistics in all data sets and then build one big dataframe having statistics about all these countries. I will save this dataframe as a csv file with the name preprocessed_country_data.csv

This will be the starting point for the clustering project.

In [251]:
print(world_happiness.shape, economy_stats.shape, public_sector_stats.shape, financial_indicators.shape, 
      health_stats.shape, population_and_environment_stats.shape, employment_stats.shape, infrastructure_stats.shape,
      crime_rates.shape)

(146, 8) (195, 4) (73, 6) (164, 3) (216, 2) (216, 6) (99, 2) (215, 3) (136, 2)


It looks like we will need to lose the public_sector_stats as it is comprised of only 73 countries and also the employment_stats as it is comprised of only 99 countries.

I will therefore use the remaining 7 data sets and see if I can get them to agree on 136 countries (the lowest number of countries between these data sets).

I will take as reference these countries and then compare them with countries in all the other datasets. After seeing the difference in all, I will see what will be the remaining countries I will go with.

In [252]:
reference_countries = set(crime_rates['country'].values)
reference_countries

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Belize',
 'Bolivia',
 'Bosnia And Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guyana',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Isle Of Man',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lebanon',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Maldives',
 'Malta',
 'Mauritius',
 'Mexico',
 'Moldova',
 'Mongolia',
 'Montenegr

In [253]:
# Difference between reference and world happiness
w_h_countries = set(world_happiness['country'].values)
print(reference_countries.difference(w_h_countries))
print(w_h_countries.difference(reference_countries))

{'Trinidad And Tobago', 'Botswana', 'Cuba', 'Czech Republic', 'Somalia', 'Sudan', 'Angola', 'Syria', 'Qatar', 'Taiwan', 'Libya', 'Luxembourg', 'Belarus', 'Hong Kong', 'Brunei', 'Kuwait', 'Isle Of Man', 'Guatemala', 'Puerto Rico', 'Azerbaijan', 'Palestine', 'Maldives', 'Oman', 'Papua New Guinea', 'Belize', 'Guyana', 'Fiji', 'Rwanda', 'Bahamas', 'Bosnia And Herzegovina'}
{'Belarus*', 'Turkmenistan*', 'Kosovo', 'Azerbaijan*', 'Benin', 'Madagascar*', 'Yemen*', 'Senegal', 'Palestinian Territories*', 'Niger*', 'Czechia', 'Guinea', 'Taiwan Province of China', 'Malawi', 'Botswana*', 'Chad*', 'Kuwait*', 'Togo', 'Mauritania*', 'Mozambique', 'Mali', 'Libya*', 'Congo', 'Hong Kong S.A.R. of China', 'Tajikistan', 'Liberia*', 'Bosnia and Herzegovina', 'Sierra Leone', 'North Cyprus*', 'Comoros*', 'Gabon', 'Ivory Coast', 'Burkina Faso', 'Rwanda*', 'Lesotho*', 'Gambia*', 'Eswatini, Kingdom of*', 'Guatemala*', 'Luxembourg*', 'Laos'}


I can see that there are lots of countries' names suffixed with a star in the world happiness dataset. I will strip that and see what the difference is afterwards.

In [254]:
world_happiness['country'] = world_happiness['country'].str.strip('*')
world_happiness['country']

RANK
1          Finland
2          Denmark
3          Iceland
4      Switzerland
5      Netherlands
          ...     
142       Botswana
143         Rwanda
144       Zimbabwe
145        Lebanon
146    Afghanistan
Name: country, Length: 146, dtype: object

In [255]:
# Difference between reference and world happiness
w_h_countries = set(world_happiness['country'].values)
print(reference_countries.difference(w_h_countries))
print(w_h_countries.difference(reference_countries))

{'Trinidad And Tobago', 'Cuba', 'Czech Republic', 'Somalia', 'Sudan', 'Angola', 'Syria', 'Qatar', 'Taiwan', 'Hong Kong', 'Brunei', 'Isle Of Man', 'Puerto Rico', 'Palestine', 'Maldives', 'Oman', 'Papua New Guinea', 'Belize', 'Guyana', 'Fiji', 'Bahamas', 'Bosnia And Herzegovina'}
{'Chad', 'Turkmenistan', 'Kosovo', 'North Cyprus', 'Benin', 'Comoros', 'Czechia', 'Guinea', 'Taiwan Province of China', 'Liberia', 'Malawi', 'Togo', 'Lesotho', 'Eswatini, Kingdom of', 'Yemen', 'Mozambique', 'Mauritania', 'Mali', 'Congo', 'Hong Kong S.A.R. of China', 'Gambia', 'Tajikistan', 'Bosnia and Herzegovina', 'Madagascar', 'Sierra Leone', 'Palestinian Territories', 'Niger', 'Gabon', 'Ivory Coast', 'Burkina Faso', 'Senegal', 'Laos'}


I can immediately see countries with the same name but different data set namings. I will create a dictionary in order to replace them.

There are 22 country names diffrent than in the world happiness country, which would bring us to 114. That is a fair number. I will manually check which ones only have the wrong namings and replace them.

In [256]:
country_namings_w_h = {'Hong Kong S.A.R. of China': 'Hong Kong', 'Taiwan Province of China': 'Taiwan', 
                   'Czechia': 'Czech Republic', 'Guinea': 'Papua New Guinea', 
                   'Bosnia and Herzegovina': 'Bosnia And Herzegovina', 'Palestinian Territories': 'Palestine'}
world_happiness.loc[
    world_happiness['country'].isin(country_namings_w_h.keys()), 'country'] = world_happiness.loc[
    world_happiness['country'].isin(country_namings_w_h.keys()), 'country'].apply(lambda x: country_namings_w_h.get(x))

In [257]:
# Difference between reference and world happiness
w_h_countries = set(world_happiness['country'].values)
print(reference_countries.difference(w_h_countries))
print(w_h_countries.difference(reference_countries))

{'Trinidad And Tobago', 'Puerto Rico', 'Maldives', 'Oman', 'Cuba', 'Belize', 'Somalia', 'Brunei', 'Sudan', 'Angola', 'Isle Of Man', 'Guyana', 'Fiji', 'Syria', 'Bahamas', 'Qatar'}
{'Chad', 'Turkmenistan', 'Kosovo', 'North Cyprus', 'Benin', 'Comoros', 'Liberia', 'Malawi', 'Togo', 'Lesotho', 'Eswatini, Kingdom of', 'Yemen', 'Mozambique', 'Mauritania', 'Mali', 'Congo', 'Gambia', 'Tajikistan', 'Madagascar', 'Sierra Leone', 'Niger', 'Gabon', 'Ivory Coast', 'Burkina Faso', 'Senegal', 'Laos'}


In [258]:
# Update reference countries with intersection
reference_countries = reference_countries.intersection(w_h_countries)
len(reference_countries)

120

In [259]:
# Difference between reference and economy stats
e_s_countries = set(economy_stats['country_name'].values)
print(sorted(reference_countries.difference(e_s_countries)))
print(sorted(e_s_countries.difference(reference_countries)))

['Bosnia And Herzegovina', 'Egypt', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Palestine', 'Russia', 'Slovakia', 'South Korea', 'Taiwan', 'Venezuela']
['American Samoa', 'Angola', 'Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Eswatini', 'Fiji', 'Gabon', 'Gambia, The', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Kiribati', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Lao PDR', 'Lesotho', 'Liberia', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Marshall Islands', 'Mauritania', 'Micronesia, Fed. Sts.', 'Monaco', 'Mozambique', 'Nauru', 'Niger', 'Oman', 'Palau', 'Puerto Rico', 'Qatar

Again we can observe different names for same countries

In [260]:
country_namings_e_s = {'Hong Kong SAR, China': 'Hong Kong', 'Iran, Islamic Rep.': 'Iran', 'Czechia': 'Czech Republic',
                       'Slovak Republic': 'Slovakia', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina',
                       'Korea, Rep.': 'South Korea', 'Kyrgyz Republic': 'Kyrgyzstan', 'Russian Federation': 'Russia',
                       'Egypt, Arab Rep.': 'Egypt'}
economy_stats.loc[
    economy_stats['country_name'].isin(country_namings_e_s.keys()), 'country_name'] = economy_stats.loc[
    economy_stats['country_name'].isin(country_namings_e_s.keys()), 'country_name'].apply(
    lambda x: country_namings_e_s.get(x))

In [261]:
# Difference between reference and economy stats
e_s_countries = set(economy_stats['country_name'].values)
print(sorted(reference_countries.difference(e_s_countries)))
print(sorted(e_s_countries.difference(reference_countries)))

['Palestine', 'Taiwan', 'Venezuela']
['American Samoa', 'Angola', 'Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eswatini', 'Fiji', 'Gabon', 'Gambia, The', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Kiribati', 'Kosovo', 'Lao PDR', 'Lesotho', 'Liberia', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Marshall Islands', 'Mauritania', 'Micronesia, Fed. Sts.', 'Monaco', 'Mozambique', 'Nauru', 'Niger', 'Oman', 'Palau', 'Puerto Rico', 'Qatar', 'Samoa', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Solomon Islands', 'Somalia', 'South Sudan', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Tajikistan', 'Timo

In [262]:
# We lose 3 more countries. We go on.
# Update reference countries with intersection
reference_countries = reference_countries.intersection(e_s_countries)
len(reference_countries)

117

In [263]:
# Difference between reference and financial indicators
f_i_countries = set(financial_indicators['country_name'].values)
print(sorted(reference_countries.difference(f_i_countries)))
print(sorted(f_i_countries.difference(reference_countries)))

['Afghanistan', 'Argentina', 'Bosnia And Herzegovina', 'Egypt', 'Ethiopia', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Libya', 'Myanmar', 'Russia', 'Slovakia', 'South Korea', 'Uzbekistan']
['Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Benin', 'Bhutan', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Congo, Rep.', "Cote d'Ivoire", 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Fiji', 'Gabon', 'Gambia, The', 'Grenada', 'Guinea', 'Guyana', 'Haiti', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Lao PDR', 'Lesotho', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Mauritania', 'Mozambique', 'Niger', 'Oman', 'Palau', 'Qatar', 'Russian Federation', 'Samoa', 'Senegal', 'Seychelles', 'Sierra Leone', 'Slovak Republic', 'Solomon Islands', 'South Sudan', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'S

In [264]:
country_namings_f_i = {'Hong Kong SAR, China': 'Hong Kong', 'Iran, Islamic Rep.': 'Iran',
                       'Slovak Republic': 'Slovakia', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina',
                       'Korea, Rep.': 'South Korea', 'Kyrgyz Republic': 'Kyrgyzstan', 'Russian Federation': 'Russia',
                       'Egypt, Arab Rep.': 'Egypt'}
financial_indicators.loc[
    financial_indicators['country_name'].isin(country_namings_f_i.keys()), 'country_name'] = financial_indicators.loc[
    financial_indicators['country_name'].isin(country_namings_f_i.keys()), 'country_name'].apply(
    lambda x: country_namings_f_i.get(x))

In [265]:
# Difference between reference and financial indicators
f_i_countries = set(financial_indicators['country_name'].values)
print(sorted(reference_countries.difference(f_i_countries)))
print(sorted(f_i_countries.difference(reference_countries)))

['Afghanistan', 'Argentina', 'Ethiopia', 'Libya', 'Myanmar', 'Uzbekistan']
['Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Benin', 'Bhutan', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Congo, Rep.', "Cote d'Ivoire", 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Fiji', 'Gabon', 'Gambia, The', 'Grenada', 'Guinea', 'Guyana', 'Haiti', 'Kosovo', 'Lao PDR', 'Lesotho', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Mauritania', 'Mozambique', 'Niger', 'Oman', 'Palau', 'Qatar', 'Samoa', 'Senegal', 'Seychelles', 'Sierra Leone', 'Solomon Islands', 'South Sudan', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Togo', 'Tonga', 'Trinidad and Tobago', 'West Bank and Gaza']


I am not ready to leave countries like Argentina and Uzbekistan outside of this analysis. I will check the other datasets and see what other countries are miissing. Perhaps we can use other countries' statistics to impute values for these 6.

In [266]:
# Difference between reference and health stats
h_s_countries = set(health_stats['country_name'].values)
print(sorted(reference_countries.difference(h_s_countries)))
print(sorted(h_s_countries.difference(reference_countries)))

['Bosnia And Herzegovina', 'Egypt', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Russia', 'Slovakia', 'South Korea']
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bosnia and Herzegovina', 'British Virgin Islands', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Isle of Man', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Lao PDR', 'Lesotho', 'Liberia', 'Liechtenstein', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldi

In [267]:
country_namings_h_s = {'Hong Kong SAR, China': 'Hong Kong', 'Iran, Islamic Rep.': 'Iran',
                       'Slovak Republic': 'Slovakia', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina',
                       'Korea, Rep.': 'South Korea', 'Kyrgyz Republic': 'Kyrgyzstan', 'Russian Federation': 'Russia',
                       'Egypt, Arab Rep.': 'Egypt'}
health_stats.loc[
    health_stats['country_name'].isin(country_namings_h_s.keys()), 'country_name'] = health_stats.loc[
    health_stats['country_name'].isin(country_namings_h_s.keys()), 'country_name'].apply(
    lambda x: country_namings_h_s.get(x))

In [268]:
# Difference between reference and health stats
h_s_countries = set(health_stats['country_name'].values)
print(sorted(reference_countries.difference(h_s_countries)))
print(sorted(h_s_countries.difference(reference_countries)))

[]
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'British Virgin Islands', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Isle of Man', 'Kiribati', "Korea, Dem. People's Rep.", 'Kosovo', 'Lao PDR', 'Lesotho', 'Liberia', 'Liechtenstein', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Marshall Islands', 'Mauritania', 'Micronesia, Fed. Sts.', 'Monaco', 'Mozambique', 'Nauru', 'New Caledonia', 'Niger', 'Northern Mariana Islands', 'Oman', 'Palau', 'Puerto Rico', 'Qatar', 'Samoa', 'San Marino', 'Sao To

In [269]:
# Difference between reference and population and environment stats
p_e_countries = set(population_and_environment_stats['country_name'].values)
print(sorted(reference_countries.difference(p_e_countries)))
print(sorted(p_e_countries.difference(reference_countries)))

['Bosnia And Herzegovina', 'Egypt', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Russia', 'Slovakia', 'South Korea']
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bosnia and Herzegovina', 'British Virgin Islands', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Isle of Man', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kyrgyz Republic', 'Lao PDR', 'Lesotho', 'Liberia', 'Liechtenstein', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Mald

In [270]:
country_namings_p_e = {'Hong Kong SAR, China': 'Hong Kong', 'Iran, Islamic Rep.': 'Iran',
                       'Slovak Republic': 'Slovakia', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina',
                       'Korea, Rep.': 'South Korea', 'Kyrgyz Republic': 'Kyrgyzstan', 'Russian Federation': 'Russia',
                       'Egypt, Arab Rep.': 'Egypt'}
population_and_environment_stats.loc[
    population_and_environment_stats['country_name'].isin(
        country_namings_p_e.keys()), 'country_name'] = population_and_environment_stats.loc[
    population_and_environment_stats['country_name'].isin(country_namings_p_e.keys()), 'country_name'].apply(
    lambda x: country_namings_p_e.get(x))

In [271]:
# Difference between reference and population and environment stats
p_e_countries = set(population_and_environment_stats['country_name'].values)
print(sorted(reference_countries.difference(p_e_countries)))
print(sorted(p_e_countries.difference(reference_countries)))

[]
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'British Virgin Islands', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Isle of Man', 'Kiribati', "Korea, Dem. People's Rep.", 'Lao PDR', 'Lesotho', 'Liberia', 'Liechtenstein', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Marshall Islands', 'Mauritania', 'Micronesia, Fed. Sts.', 'Monaco', 'Mozambique', 'Nauru', 'New Caledonia', 'Niger', 'Northern Mariana Islands', 'Oman', 'Palau', 'Puerto Rico', 'Qatar', 'Samoa', 'San Marino', 'Sao T

In [272]:
# Difference between reference and infrastructure stats
i_s_countries = set(infrastructure_stats['country_name'].values)
print(sorted(reference_countries.difference(i_s_countries)))
print(sorted(i_s_countries.difference(reference_countries)))

['Bosnia And Herzegovina', 'Egypt', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Russia', 'Slovakia', 'South Korea']
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bosnia and Herzegovina', 'British Virgin Islands', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Isle of Man', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kyrgyz Republic', 'Lao PDR', 'Lesotho', 'Liberia', 'Liechtenstein', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Mar

In [273]:
country_namings_i_s = {'Hong Kong SAR, China': 'Hong Kong', 'Iran, Islamic Rep.': 'Iran',
                       'Slovak Republic': 'Slovakia', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina',
                       'Korea, Rep.': 'South Korea', 'Kyrgyz Republic': 'Kyrgyzstan', 'Russian Federation': 'Russia',
                       'Egypt, Arab Rep.': 'Egypt'}
infrastructure_stats.loc[
    infrastructure_stats['country_name'].isin(
        country_namings_i_s.keys()), 'country_name'] = infrastructure_stats.loc[
    infrastructure_stats['country_name'].isin(country_namings_i_s.keys()), 'country_name'].apply(
    lambda x: country_namings_i_s.get(x))

In [274]:
# Difference between reference and infrastructure stats
i_s_countries = set(infrastructure_stats['country_name'].values)
print(sorted(reference_countries.difference(i_s_countries)))
print(sorted(i_s_countries.difference(reference_countries)))

[]
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'British Virgin Islands', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Isle of Man', 'Kiribati', "Korea, Dem. People's Rep.", 'Lao PDR', 'Lesotho', 'Liberia', 'Liechtenstein', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Marshall Islands', 'Mauritania', 'Micronesia, Fed. Sts.', 'Monaco', 'Mozambique', 'Nauru', 'New Caledonia', 'Niger', 'Northern Mariana Islands', 'Oman', 'Palau', 'Puerto Rico', 'Qatar', 'Samoa', 'San Marino', 'Sao Tome and Principe', 

It looks like for all the other data sets we have the values for all the 117 countries. I will attempt to fill out the financial indicators for the 6 countries individually.

I cannot simply replace the values here, since I already preprocessed them. Therefore, for this particular dataset I will reiterate the preprocessing steps, but add the 6 needed rows for ['Afghanistan', 'Argentina', 'Ethiopia', 'Libya', 'Myanmar', 'Uzbekistan'] before scaling.

In [275]:
# Reiterate exact steps, but complete dataset with values for 
# ['Afghanistan', 'Argentina', 'Ethiopia', 'Libya', 'Myanmar', 'Uzbekistan'] before scaling

financial_indicators = pd.read_excel("../data/financial-indicators.xlsx", na_values="..")
financial_indicators = preprocess_world_bank_data(financial_indicators)

# Complete with values found on the web
financial_indicators.loc[10, 'Consumer price index (2010 = 100)'] = 212.3
financial_indicators.loc[10, 'Inflation, consumer prices (annual %)'] = 4.1003

financial_indicators.loc[27, 'Consumer price index (2010 = 100)'] = 121
financial_indicators.loc[27, 'Inflation, consumer prices (annual %)'] = -0.6

financial_indicators.loc[68, 'Consumer price index (2010 = 100)'] = 415
financial_indicators.loc[68, 'Inflation, consumer prices (annual %)'] = 39.9

financial_indicators.loc[109, 'Consumer price index (2010 = 100)'] = 439
financial_indicators.loc[109, 'Inflation, consumer prices (annual %)'] = 11.4

# Drop unuseful column
financial_indicators = financial_indicators.drop(columns="Listed domestic companies, total")

financial_indicators.head()

series_name,country_name,Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)"
0,Albania,123.448662,2.041472
1,Algeria,166.218895,7.226063
2,Antigua and Barbuda,118.555595,2.062996
3,Armenia,140.134411,7.184836
4,Australia,124.271592,2.863910


In [276]:
# Fill with missing values
financial_indicators.loc[300, :] = ['Afghanistan', 149.896, 2.3]
financial_indicators.loc[301, :] = ['Argentina', 121, 53.55]
financial_indicators.loc[302, :] = ['Ethiopia', 319, 15.84]
financial_indicators.loc[303, :] = ['Libya', 189.66, 2.416]
financial_indicators.loc[304, :] = ['Myanmar', 165, 9.113]
financial_indicators.loc[305, :] = ['Uzbekistan', 207.66, 12.9]

financial_indicators.tail()

series_name,country_name,Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)"
301,Argentina,121.00,53.550
302,Ethiopia,319.00,15.840
303,Libya,189.66,2.416
304,Myanmar,165.00,9.113
305,Uzbekistan,207.66,12.900


In [277]:
# Get numerical columns for the world happiness dataset
numerical_columns_f_i = financial_indicators.columns.copy().drop("country_name")

# Again, I will use scaling so that all features keep their impact respective to their groups
financial_indicators[numerical_columns_f_i] = data_scaler.fit_transform(financial_indicators[numerical_columns_f_i])

# Reverse statistic so high-value = good, low-value = bad; also rename column
financial_indicators[numerical_columns_f_i] = 1.0 - financial_indicators[numerical_columns_f_i]
financial_indicators.rename(columns={'Consumer price index (2010 = 100)': 'inverse_of_consumer_price_', 
                                     'Inflation, consumer prices (annual %)': 'inverse_of_inflation'})
financial_indicators.head()

series_name,country_name,Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)"
0,Albania,0.998817,0.992632
1,Algeria,0.996712,0.983362
2,Antigua and Barbuda,0.999057,0.992593
3,Armenia,0.997996,0.983435
4,Australia,0.998776,0.991161


In [278]:
# Difference between reference and financial indicators
f_i_countries = set(financial_indicators['country_name'].values)
print(sorted(reference_countries.difference(f_i_countries)))
print(sorted(f_i_countries.difference(reference_countries)))

['Bosnia And Herzegovina', 'Egypt', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Russia', 'Slovakia', 'South Korea']
['Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Benin', 'Bhutan', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Congo, Rep.', "Cote d'Ivoire", 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Fiji', 'Gabon', 'Gambia, The', 'Grenada', 'Guinea', 'Guyana', 'Haiti', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Lao PDR', 'Lesotho', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Mauritania', 'Mozambique', 'Niger', 'Oman', 'Palau', 'Qatar', 'Russian Federation', 'Samoa', 'Senegal', 'Seychelles', 'Sierra Leone', 'Slovak Republic', 'Solomon Islands', 'South Sudan', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Togo', 'Tonga', 'Trinidad and Tobago', 'West Bank and 

In [279]:
country_namings_f_i = {'Hong Kong SAR, China': 'Hong Kong', 'Iran, Islamic Rep.': 'Iran',
                       'Slovak Republic': 'Slovakia', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina',
                       'Korea, Rep.': 'South Korea', 'Kyrgyz Republic': 'Kyrgyzstan', 'Russian Federation': 'Russia',
                       'Egypt, Arab Rep.': 'Egypt'}
financial_indicators.loc[
    financial_indicators['country_name'].isin(country_namings_f_i.keys()), 'country_name'] = financial_indicators.loc[
    financial_indicators['country_name'].isin(country_namings_f_i.keys()), 'country_name'].apply(
    lambda x: country_namings_f_i.get(x))

In [280]:
# Difference between reference and financial indicators
f_i_countries = set(financial_indicators['country_name'].values)
print(sorted(reference_countries.difference(f_i_countries)))
print(sorted(f_i_countries.difference(reference_countries)))

[]
['Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Benin', 'Bhutan', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Congo, Rep.', "Cote d'Ivoire", 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Fiji', 'Gabon', 'Gambia, The', 'Grenada', 'Guinea', 'Guyana', 'Haiti', 'Kosovo', 'Lao PDR', 'Lesotho', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Mauritania', 'Mozambique', 'Niger', 'Oman', 'Palau', 'Qatar', 'Samoa', 'Senegal', 'Seychelles', 'Sierra Leone', 'Solomon Islands', 'South Sudan', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Togo', 'Tonga', 'Trinidad and Tobago', 'West Bank and Gaza']


Perfect, I now have my total number of countries which will go into this app: 117.
Now I can compile all the data into 1 dataframe and save it so I can use it in the clustering part.

In [281]:
# Get final datasets with the same countries
world_happiness_final = world_happiness.loc[world_happiness['country'].isin(reference_countries)]
crime_rates_final = crime_rates.loc[crime_rates['country'].isin(reference_countries)]
economy_stats_final = economy_stats.loc[economy_stats['country_name'].isin(reference_countries)].rename(
    columns={'country_name': 'country'})
health_stats_final = health_stats.loc[health_stats['country_name'].isin(reference_countries)].rename(
    columns={'country_name': 'country'})
infrastructure_stats_final = infrastructure_stats.loc[
    infrastructure_stats['country_name'].isin(reference_countries)].rename(columns={'country_name': 'country'})
financial_indicators_final = financial_indicators.loc[
    financial_indicators['country_name'].isin(reference_countries)].rename(columns={'country_name': 'country'})
population_and_environment_stats_final = population_and_environment_stats.loc[
    population_and_environment_stats['country_name'].isin(reference_countries)].rename(columns={'country_name': 'country'})

In [282]:
# Combine all datasets into one
cleaned_dataset = world_happiness_final.merge(
    crime_rates_final, on='country', how='inner').merge(
    economy_stats_final, on='country', how='inner').merge(
    health_stats_final, on='country', how='inner').merge(
    infrastructure_stats_final, on='country', how='inner').merge(
    financial_indicators_final, on='country', how='inner').merge(
    population_and_environment_stats_final, on='country', how='inner')
cleaned_dataset.head()

,country,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,opposite_of_crime_index,Adjusted net national income (current US$),...,Population growth (annual %),Fixed broadband subscriptions (per 100 people),Secure Internet servers (per 1 million people),Consumer price index (2010 = 100),"Inflation, consumer prices (annual %)",Access to electricity (% of population),Forest area (% of land area),Land area (sq. km),Population density (people per sq. km of land area),Urban population (% of total population)
0,Finland,1.000000,0.856496,0.953030,0.822718,0.994595,0.232906,0.909710,0.784169,0.012505,...,0.334909,0.534286,0.110178,0.999226,0.992358,1.0,0.756920,0.018558,0.000915,0.85517
1,Denmark,0.965848,0.884110,0.941667,0.824841,0.971622,0.401709,0.906303,0.803295,0.017378,...,0.362043,0.717153,0.373890,0.999337,0.992969,1.0,0.161284,0.002442,0.007380,0.88116
2,Iceland,0.951265,0.876415,1.000000,0.852442,0.970270,0.576923,0.325383,0.837777,0.000978,...,0.606145,0.666454,0.101897,0.998072,0.988336,1.0,0.005228,0.006157,0.000177,0.93898
3,Switzerland,0.942957,0.917157,0.928788,0.872611,0.914865,0.314103,0.785349,0.867514,0.031193,...,0.439125,0.746407,0.161959,1.000000,0.995242,1.0,0.329695,0.002413,0.011067,0.73915
4,Netherlands,0.925051,0.880489,0.913636,0.835456,0.879730,0.579060,0.713799,0.790172,0.041755,...,0.410425,0.704263,0.184681,0.998961,0.991498,1.0,0.112657,0.002056,0.026239,0.92236


In [283]:
# Rename some of the columns
cleaned_dataset = cleaned_dataset.rename(columns=lambda c: re.sub(r' ', '_', re.sub(r' \(.*\)', '', c)).lower())
cleaned_dataset.head()

,country,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,opposite_of_crime_index,adjusted_net_national_income,...,population_growth,fixed_broadband_subscriptions,secure_internet_servers,consumer_price_index,"inflation,_consumer_prices",access_to_electricity,forest_area,land_area,population_density,urban_population
0,Finland,1.000000,0.856496,0.953030,0.822718,0.994595,0.232906,0.909710,0.784169,0.012505,...,0.334909,0.534286,0.110178,0.999226,0.992358,1.0,0.756920,0.018558,0.000915,0.85517
1,Denmark,0.965848,0.884110,0.941667,0.824841,0.971622,0.401709,0.906303,0.803295,0.017378,...,0.362043,0.717153,0.373890,0.999337,0.992969,1.0,0.161284,0.002442,0.007380,0.88116
2,Iceland,0.951265,0.876415,1.000000,0.852442,0.970270,0.576923,0.325383,0.837777,0.000978,...,0.606145,0.666454,0.101897,0.998072,0.988336,1.0,0.005228,0.006157,0.000177,0.93898
3,Switzerland,0.942957,0.917157,0.928788,0.872611,0.914865,0.314103,0.785349,0.867514,0.031193,...,0.439125,0.746407,0.161959,1.000000,0.995242,1.0,0.329695,0.002413,0.011067,0.73915
4,Netherlands,0.925051,0.880489,0.913636,0.835456,0.879730,0.579060,0.713799,0.790172,0.041755,...,0.410425,0.704263,0.184681,0.998961,0.991498,1.0,0.112657,0.002056,0.026239,0.92236


In [284]:
# Save the dataframe into a csv file to be used in the next notebook
# I will run this notebook twice: once with a standard scaled dataset and one with min_max, to check the impact scaling
# has on clustering
cleaned_dataset.to_csv('../data/cleaned_data' + '_' + data_scaler_type + '.csv', index=False)